# **RESPONSI PRAKTIKUM ANALITIKA DATA**
**Editha Anggrieniputri Kinbenu (19/445626/PA/19450)**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.random import set_seed

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from numpy import array
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/responsiandat/ETH-USD.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
plt.plot(df['Close'])

In [ ]:
df1=df['Close']

In [ ]:
df1.head()

In [ ]:
df1.head()

In [ ]:
df1.tail()

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
df1 = scaler.fit_transform(np.array(df1).reshape(-1,1))

In [ ]:
#splitting data menjadi data train dan data test
training_size = int(len(df1)*0.836066)
test_size = len(df1) - training_size
train_data, test_data = df1[0:,:],df1[training_size:len(df1),:]
time_train = df.iloc[:training_size,0]
time_valid = df.iloc[training_size:,0]

In [ ]:
plt.plot(train_data)

In [ ]:
plt.plot(test_data)

In [ ]:
training_size, test_size

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
fig, ax = plt.subplots(2, figsize=(10,6))
ax[0]=plot_acf(df.iloc[:,4],ax=ax[0])
ax[1]=plot_pacf(df.iloc[:,4],ax=ax[1])

In [ ]:
#Mengubah array menjadi data matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-11):
        a = dataset[i:(i+time_step),0]
        dataX.append(a)
        dataY.append(dataset[i+time_step,0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step=3
X_train, Y_train = create_dataset(train_data, time_step)
X_test, Y_test = create_dataset(test_data, time_step)

In [ ]:
print(X_train.shape), print(Y_train.shape)

In [ ]:
#reshape input
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
np.random.seed(123)
set_seed(123)

In [ ]:
model = Sequential()
model.add(LSTM(200, input_shape=(None,1)))
model.add(Dense(5,activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','mape'])
model.summary()
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='LSTM model.png', show_shapes=True)

In [ ]:
history = model.fit(X_train, Y_train, validation_split=0.1, epochs=25, batch_size=64, verbose=1)

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

loss=history.history['loss']
val_loss=history.history['val_loss']
mae=history.history['mae']
val_mae=history.history['val_mae']
mape=history.history['mape']
val_mape=history.history['val_mape']

epochs=range(len(loss))

plt.plot(epochs, loss, 'r', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label ='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, mae, 'r', label = 'Training MAE')
plt.plot(epochs, val_mae, 'b', label ='Validation MAE')
plt.title('Training and validation MAE')
plt.legend()
plt.figure()

plt.plot(epochs, mape, 'r', label = 'Training MAPE')
plt.plot(epochs, val_mape, 'b', label ='Validation MAPE')
plt.title('Training and validation MAPE')
plt.legend()
plt.figure()

In [ ]:
#loss, MAPE, MSE
test_eval=model.evaluate(X_test,Y_test)
for name, value in zip(model.metrics_names, test_eval):
    print(name,': ',value)
print()

In [ ]:
test_predict = model.predict(X_test)

In [ ]:
test_predict1 = scaler.inverse_transform(test_predict)

In [ ]:
#RMSE
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(Y_test, test_predict))

In [ ]:
train = df[:training_size+3]
valid = df[training_size+3:len(df1)-11]
valid['Predictions'] = test_predict1
plt.figure(figsize=(16,8))
plt.title('Model LSTM', fontsize=18)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Value', fontsize=12)
plt.plot(valid[['Close', 'Predictions']])
plt.legend([ 'Val', 'Predictions'], loc='lower right')
plt.show()
print(test_predict1)